In [4]:
data = []
with open('relations.csv') as f:
    data = [row[:-1].split(',') for row in f][1:]

In [25]:
def process_r(r):
    r = r.replace('-', '_').replace('/', '_or_').replace(' ', '_')
    char_lst=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    
    res = ""
    for i in r:
        if i == '_':
            res += i
        else:
            if i in char_lst:
                res += i
                
    return res.upper()

In [28]:
with open('visulization.cypher', 'w') as f:
    f.write('LOAD CSV WITH HEADERS FROM "file:///nodes.csv" AS line\n')
    f.write('MERGE (p:person{id:line.id, name:line.name, blood_status:line.blood_status, hair_color:line.hair_color, eye_color:line.eye_color, patronus:line.patronus, wand:line.wand, occupation:line.occupation, loyalty:line.loyalty, house:line.house});\n\n')

    for row in data:
        f.write("MATCH (from:person{id:'"+row[0]+"'}),(to:person{id:'"+row[2]+"'})\n")
        f.write("MERGE (from)-[r:{}]->(to);\n".format(process_r(row[1])))

In [5]:
data

[['1', 'daughter', '58'],
 ['1', 'father-in-law', '24'],
 ['1', 'nephew/godson', '6'],
 ['1', 'nephew', '175'],
 ['1', 'niece', '4'],
 ['1', 'brother-in-law', '17'],
 ['4', 'paternal grandfather', '8'],
 ['4', 'maternal grandfather', '24'],
 ['4', 'maternal uncle', '20'],
 ['4', 'maternal uncle', '18'],
 ['4', 'maternal uncle', '17'],
 ['4', 'maternal first cousin', '58'],
 ['5', 'father-in-law', '7'],
 ['6', 'paternal great-grandfather', '7'],
 ['6', 'maternal grandfather', '24'],
 ['6', 'maternal first cousin', '58'],
 ['6', 'maternal uncle', '17'],
 ['6', 'maternal uncle', '20'],
 ['7', 'son', '8'],
 ['8', 'father', '7'],
 ['9', 'paternal grandfather', '7'],
 ['9', 'father', '8'],
 ['9', 'father-in-law', '24'],
 ['9', 'brother-in-law', '20'],
 ['9', 'brother-in-law', '17'],
 ['9', 'niece', '58'],
 ['12', 'father', '24'],
 ['12', 'younger brother', '20'],
 ['12', 'younger brother', '18'],
 ['12', 'younger brother', '17'],
 ['12', 'niece', '58'],
 ['13', 'father', '24'],
 ['13', 'olde